In [ ]:
import os
import streamlit as st
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
load_dotenv()
# Variáveis de ambiente
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = 'teste-gpt-4o'
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')
index_name = 'index-1'  # Nome do índice no Azure Search
admin_key = ''

# Inicializa o cliente do Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version='2024-02-01',
    azure_endpoint=azure_endpoint
)

# Função de busca no Azure Search
def search_example(query_text, top_n):
    search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(admin_key))
    results = search_client.search(query_text, top=top_n)
    return [{"documento": result["documento"], "pagina": result["pagina"], "score": result["@search.score"], 'texto': result['texto']} for result in results]

# Prompt único que será usado tanto na busca quanto no chat completions

prompt = 'me de documentos que indiquem ressarcimento com contratos da empresa X '

# Realiza a busca e formata o conteúdo
top_n = 5
search_results = search_example(prompt, top_n)

# Concatena os resultados da busca em um texto único
contexto = "\n\n".join([f"Documento: {result['documento']}, Página: {result['pagina']}\nTexto: {result['texto']}" for result in search_results])

# Prepara o prompt e envia para o Azure OpenAI

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Responda sempre em português e seja o mais fiel possivel ao conteudo original"
        },
        {
            "role": "user",
            "content": f"Com base no contexto a seguir: {contexto}, {prompt}",
        }
    ],
    temperature=0.1,
    model=deployment_name,

    )

response_content = response.choices[0].message.content
print(response_content)
